In [1]:
import ID3_with_continuous_feature_support_exp

Sepal_L
23
Petal_L
6
['Sepal_L', 'Sepal_W', 'Petal_L', 'Petal_W'] ['setosa', 'versicolor', 'virginica']
       Sepal_L  Sepal_W      Petal_L  Petal_W
0  Sepal_L<5.6      3.5  Petal_L<3.0      0.2
1  Sepal_L<5.6      3.0  Petal_L<3.0      0.2
2  Sepal_L<5.6      3.2  Petal_L<3.0      0.2
3  Sepal_L<5.6      3.1  Petal_L<3.0      0.2
4  Sepal_L<5.6      3.6  Petal_L<3.0      0.2
5  Sepal_L<5.6      3.9  Petal_L<3.0      0.4
6  Sepal_L<5.6      3.4  Petal_L<3.0      0.3
7  Sepal_L<5.6      3.4  Petal_L<3.0      0.2
8  Sepal_L<5.6      2.9  Petal_L<3.0      0.2
9  Sepal_L<5.6      3.1  Petal_L<3.0      0.1


Press Enter to continue: 


attr_name Petal_W
labels [0.2, 0.4, 0.3, 0.1, 0.5, 0.6, 1.4, 1.5, 1.3, 1.6, 1.0, 1.1, 1.8, 1.2, 1.7, 2.5, 1.9, 2.1, 2.2, 2.0, 2.4, 2.3]
attr_name Sepal_W
labels [3.2, 3.1, 2.8, 3.0, 2.2, 2.5, 2.9]
attr_name Sepal_L
labels ['Sepal_L>=5.6']
attr_name Petal_L
labels ['Petal_L>=3.0']
attr_name Sepal_W
labels [3.2, 2.9, 2.5, 3.0]
attr_name Sepal_L
labels ['Sepal_L>=5.6', 'Sepal_L<5.6']
['Petal_W', 0.2, 'Species', 'setosa']
['Petal_W', 0.4, 'Species', 'setosa']
['Petal_W', 0.3, 'Species', 'setosa']
['Petal_W', 0.1, 'Species', 'setosa']
['Petal_W', 0.5, 'Species', 'setosa']
['Petal_W', 0.6, 'Species', 'setosa']
['Petal_W', 1.4, 'Species', 'versicolor']
['Petal_W', 1.5, 'Sepal_W', 3.2, 'Species', 'versicolor']
['Petal_W', 1.5, 'Sepal_W', 3.1, 'Species', 'versicolor']
['Petal_W', 1.5, 'Sepal_W', 2.8, 'Species', 'versicolor']
['Petal_W', 1.5, 'Sepal_W', 3.0, 'Species', 'versicolor']
['Petal_W', 1.5, 'Sepal_W', 2.2, 'Sepal_L', 'Sepal_L>=5.6', 'Petal_L', 'Petal_L>=3.0', 'Species', 'versicolor']
['

Press Enter for reduced error pruning 


['Petal_W', 0.2, 'Species', 'setosa']
setosa precision 0 recall 0
versicolor precision 0 recall 0.0
virginica precision 0.8518518518518519 recall 1.0
Summary :  correct: 23 total : 30 accuracy : 76.66666666666667 %
76.66666666666667
['Petal_W', 0.4, 'Species', 'setosa']
setosa precision 0 recall 0
versicolor precision 0 recall 0.0
virginica precision 0.8518518518518519 recall 1.0
Summary :  correct: 23 total : 30 accuracy : 76.66666666666667 %
76.66666666666667
['Petal_W', 0.3, 'Species', 'setosa']
setosa precision 0 recall 0
versicolor precision 0 recall 0.0
virginica precision 0.8518518518518519 recall 1.0
Summary :  correct: 23 total : 30 accuracy : 76.66666666666667 %
76.66666666666667
['Petal_W', 0.1, 'Species', 'setosa']
setosa precision 0 recall 0
versicolor precision 0 recall 0.0
virginica precision 0.8518518518518519 recall 1.0
Summary :  correct: 23 total : 30 accuracy : 76.66666666666667 %
76.66666666666667
['Petal_W', 0.5, 'Species', 'setosa']
setosa precision 0 recall 0
ve